In [77]:
import requests
import bs4
from bs4 import BeautifulSoup as soup
import json
import pandas
from pandas import DataFrame
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy
from collections import defaultdict
nlp = spacy.load('en_core_web_md')
from string import punctuation
import itertools

In [78]:
from collections import defaultdict

In [79]:
def get_ld_json(url: str) -> dict:
    parser = "html.parser"
    req = requests.get(url)
    page = soup(req.text, parser)
    return json.loads("".join(page.find("script", {"type":"application/ld+json"}).contents))
def get_ingredientScript(url):
    jsonld = get_ld_json(url)
    useful = jsonld[1]
    ingredients = useful["recipeIngredient"]
    return ingredients
def parse_ingredients(url):
    ingredients = get_ingredientScript(url)
    df = DataFrame (ingredients,columns=['ingredients'])
    # 2 cases
    # "number" ("()") ("unit") (adjective) "noun/subject - ingredient" (, other)
    # contains "to taste"
    df = df["ingredients"]
    df_taste = df[df.str.contains('to taste', case = False)]
    
    df_unit = df[~df.str.contains('to taste', case = False)]
    
    # array of arrays: each array is amount, unit, ingredient, descriptor, preparation
    ingredients_parsed = []
    
    for i in df:
        curr_arr = ["", "", "", "", ""]
        print(i)
        if 'to taste' not in i:
        
            # before we look at POS, remove everything after the comma and put it in prep
            split_string = i.split(", ", 1)
            root_phrase = split_string[0]
            if len(split_string) > 1:
                other_piece = split_string[1]
            else:
                other_piece = ""

            curr_arr[4] = other_piece

            split_string2 = root_phrase.split("(", 1)
            if len(split_string2) > 1:
                split_string3 = split_string2[1].split(")", 1)
                curr_arr[3] = split_string3[0]
                root_phrase = split_string2[0].strip() + split_string3[1]
            doc = nlp(root_phrase)
            index = 0
            for token in doc:
                found_num = False
                # only get first number if it matches criteria
                #if found_num == False and token.pos_ == "NOUN" and not token.is_alpha or token.pos_ == "NUM":
                if index == 0:
                    curr_arr[0] = token.text
                elif index == 1:
                    if token.pos_ != "ADJ":
                        curr_arr[1] = token.text
                    else:
                        curr_arr[3] = token.text
                elif token.dep_ == "ROOT":
                    curr_arr[2] = token.text
                else: 
                    curr_arr[3] = curr_arr[3] + " " + token.text
                    curr_arr[3] = curr_arr[3].strip()
                index+=1
        else:
            i = i.replace("to taste", "")
            doc = nlp(i)
            curr_arr[0] = "to taste"
            for token in doc:
                if token.dep_ == "ROOT":
                    curr_arr[2] = token.text
                else:
                    curr_arr[3] = curr_arr[3] + " " + token.text
                    curr_arr[3] = curr_arr[3].strip()
        ingredients_parsed.append(curr_arr)
        #print(token.text, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop, token.children, token.head)
        #displacy.render(doc, style="dep") # change to serve when we go to python


    
    return ingredients_parsed

In [80]:
url1 = "https://www.allrecipes.com/recipe/232550"

In [81]:
json_ld = get_ld_json(url1)
print(json_ld)

[{'@context': 'http://schema.org', '@type': 'BreadcrumbList', 'itemListElement': [{'@type': 'ListItem', 'position': 1, 'item': {'@id': 'https://www.allrecipes.com/', 'name': 'Home', 'image': None}}, {'@type': 'ListItem', 'position': 2, 'item': {'@id': 'https://www.allrecipes.com/recipes/', 'name': 'Recipes', 'image': None}}, {'@type': 'ListItem', 'position': 3, 'item': {'@id': 'https://www.allrecipes.com/recipes/78/breakfast-and-brunch/', 'name': 'Breakfast and Brunch Recipes', 'image': None}}, {'@type': 'ListItem', 'position': 4, 'item': {'@id': 'https://www.allrecipes.com/recipes/1304/breakfast-and-brunch/breakfast-bread/', 'name': 'Breakfast Bread Recipes', 'image': None}}]}, {'@context': 'http://schema.org', '@type': 'Recipe', 'mainEntityOfPage': 'https://www.allrecipes.com/recipe/232550/pumpkin-sweet-potato-bread/', 'name': 'Pumpkin Sweet Potato Bread', 'image': {'@type': 'ImageObject', 'url': 'https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fimages.media-allrecipes.

In [82]:
ing = parse_ingredients(url1)
print(ing)

1 cup pumpkin pie filling
1 cup sweet potato pie filling
½ cup water
½ cup vegetable oil
3 cups all-purpose flour
1 teaspoon baking soda
1 teaspoon baking powder
[['1', 'cup', 'filling', 'pumpkin pie', ''], ['1', 'cup', 'filling', 'sweet potato pie', ''], ['½', 'cup', 'water', '', ''], ['½', 'cup', '', 'vegetable oil', ''], ['3', 'cups', '', 'all - purpose flour', ''], ['1', 'teaspoon', 'soda', 'baking', ''], ['1', 'teaspoon', 'powder', 'baking', '']]


In [83]:
one_list = itertools.chain.from_iterable(ing)
fil = list(dict.fromkeys(one_list))
new = []
for ing in fil:
    for word in ing.split():
        new.append(word)
print(new)

['1', 'cup', 'filling', 'pumpkin', 'pie', 'sweet', 'potato', 'pie', '½', 'water', 'vegetable', 'oil', '3', 'cups', 'all', '-', 'purpose', 'flour', 'teaspoon', 'soda', 'baking', 'powder']


NameError: name 'quanti' is not defined

In [84]:
text = json_ld[1]['recipeInstructions']
if len(text) > 1:
    new_text = ''
    for i,t in enumerate(text):
#         print(text[i]['text'])
        new_text = new_text + text[i]['text']
    text = new_text
else:
    text = text[0]['text']
print(text)

Preheat oven to 325 degrees F (165 degrees C). Grease 2 loaf pans.
Mix pumpkin pie filling, sweet potato pie filling, water, and oil together in a bowl; add flour, baking soda, and baking powder. Stir well. Pour mixture into the prepared loaf pans.
Bake in the preheated oven until a knife inserted into each loaf comes out clean, about 1 hour.



In [85]:
doc = nlp('Pour mixture into the prepared loaf pans')
for token in doc:
    print(token.text,token.pos_, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])
displacy.render(doc, style="dep")

Pour VERB ROOT Pour VERB [mixture, into]
mixture NOUN dobj Pour VERB []
into ADP prep Pour VERB [pans]
the DET det pans NOUN []
prepared ADJ amod pans NOUN []
loaf NOUN compound pans NOUN []
pans NOUN pobj into ADP [the, prepared, loaf]


In [86]:
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)

mixture mixture dobj Pour
the prepared loaf pans pans pobj into


### Thing that could be tools
#### 1) pattern: lemma: 'in',   

In [87]:
def find_tool(sentence):
    pattern = [{"LEMMA":{"IN":['in','with']}},{"POS":'DET'},{"POS":'NOUN', 'OP':'+'}]
    # tool_pattern = [{"POS":'NOUN', 'OP':'+'}]
    tools = []
    matcher = Matcher(nlp.vocab)
    matcher.add('find', [pattern])
    doc = nlp(sentence)
    matches = matcher(doc)
    if len(matches) != 0:
        span = doc[matches[-1][1]:matches[-1][2]]
        for token in span:
            if token.pos_ == 'NOUN':
                tools.append(token)
    return tools
find_tool('Heat butter in a pan over medium heat')
def find_tool2(sentence):
    too =[]
    not_tools = []
    sent = nlp(sentence)
    for chunk in sent.noun_chunks:
        if chunk.root.text not in new:
            too.append(chunk.root.text)
    for token in sent:
        if token.text not in too and token.pos_ == 'NOUN':
            not_tools.append(token.text)
    return too, not_tools
        
    

In [88]:
tools = []
big_not_tools = {}

for sentence in nlp(text).sents:
    print(sentence)
    t,nt = find_tool2(str(sentence))
    for each in t:
        print(each)
        if each not in big_not_tools:
            tools.append(each)
    for word in nt:
        big_not_tools[word] = 0
print(tools)
    
    
    

Preheat oven to 325 degrees F (165 degrees C).
Preheat
F
C
Grease 2 loaf pans.

Grease
pans
Mix pumpkin pie filling, sweet potato pie filling, water, and oil together in a bowl; add flour, baking soda, and baking powder.
bowl
Stir well.
Pour mixture into the prepared loaf pans.

mixture
pans
Bake in the preheated oven until a knife inserted into each loaf comes out clean, about 1 hour.

oven
knife
loaf
['Preheat', 'F', 'C', 'Grease', 'pans', 'bowl', 'mixture', 'pans', 'oven', 'knife']
